<a href="https://colab.research.google.com/github/ipeirotis-org/datasets/blob/main/NYC_Traffic_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import geopandas as gpd
from shapely import wkt

In [ ]:
!curl 'https://data.cityofnewyork.us/api/views/7ym2-wayt/rows.csv?accessType=DOWNLOAD' -o traffic.csv

In [ ]:
# Load the dataset
df = pd.read_csv("traffic.csv")

# Combine the date and time columns into a single datetime column
df['DateTime'] = pd.to_datetime(df[['Yr', 'M', 'D', 'HH', 'MM']].astype(str).agg('-'.join, axis=1), format='%Y-%m-%d-%H-%M')

# Drop the original date and time columns if you no longer need them
df = df.drop(columns=['Yr', 'M', 'D', 'HH', 'MM'])

In [ ]:
# Convert WktGeom to geometry object
df['geometry'] = df['WktGeom'].apply(wkt.loads)

# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:2263')

# Drop the original WktGeom column if you no longer need it
gdf = gdf.drop(columns=['WktGeom'])


In [ ]:
gdf_wgs84 = gdf.to_crs('EPSG:4326')
gdf_wgs84

In [ ]:
gdf_wgs84

In [ ]:
pvt = gdf.pivot_table(
    index='DateTime',
    values='Vol',
    columns='SegmentID',
    aggfunc='mean'
).resample('1Y').mean()

